In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import requests
import time
import os
import json
import csv
import urllib.parse
import re
from datetime import datetime
from PIL import Image
import io

# =================== 설정 구역 ===================
# 여기서 검색어와 기타 설정을 변경하세요
SEARCH_KEYWORD = "신림동 햄버거"  # 원하는 검색어로 변경
MAX_RESTAURANTS = 1  # 크롤링할 최대 가게 수
MAX_REVIEWS_PER_RESTAURANT = 10  # 가게당 최대 리뷰 수
# ================================================

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def create_directory_structure(keyword):
    """가게별 이미지 저장용 디렉토리 구조 생성"""
    base_dir = f"{keyword}_테스트(1개)_data"
    os.makedirs(base_dir, exist_ok=True)
    return base_dir

def create_restaurant_directories(base_dir, restaurant_name):
    """가게별 폴더 구조 생성"""
    safe_name = safe_filename(restaurant_name)
    restaurant_dir = os.path.join(base_dir, safe_name)
    menu_images_dir = os.path.join(restaurant_dir, "메뉴_이미지")
    review_images_dir = os.path.join(restaurant_dir, "리뷰_이미지")
    
    os.makedirs(menu_images_dir, exist_ok=True)
    os.makedirs(review_images_dir, exist_ok=True)
    
    return restaurant_dir, menu_images_dir, review_images_dir

def download_image_improved(url, filename, images_dir):
    """이미지 다운로드 (개선된 버전 - 이미지 검증 포함)"""
    try:
        # User-Agent 헤더 추가
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
            'Referer': 'https://map.naver.com/'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            # 이미지 데이터 검증
            try:
                # PIL로 이미지 검증
                img = Image.open(io.BytesIO(response.content))
                img.verify()  # 이미지가 유효한지 검증
                
                # 파일 저장
                filepath = os.path.join(images_dir, filename)
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                print(f"  이미지 다운로드 성공: {filename}")
                return filepath
            except Exception as img_error:
                print(f"  이미지 검증 실패 ({filename}): {img_error}")
                return None
        else:
            print(f"  HTTP 오류 ({filename}): {response.status_code}")
            return None
    except Exception as e:
        print(f"  이미지 다운로드 실패 ({filename}): {e}")
        return None

def safe_filename(filename):
    """파일명에서 특수문자 제거"""
    return re.sub(r'[^\w\s-]', '', filename).strip()[:50]

def get_restaurant_list():
    """왼쪽 패널에서 가게 리스트 수집"""
    print("가게 리스트 수집 중...")
    
    # 스크롤하여 모든 가게 로드
    try:
        scroll_container = driver.find_element(By.CSS_SELECTOR, "div.Ryr1F")
        print("스크롤 컨테이너 찾음")
        
        previous_count = 0
        max_attempts = 15
        
        for attempt in range(max_attempts):
            restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(restaurant_elements)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            if current_count == previous_count:
                if attempt >= 3:
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                previous_count = current_count
            
            # 스크롤 실행
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            time.sleep(2)
        
        return restaurant_elements
        
    except Exception as e:
        print(f"가게 리스트 수집 실패: {e}")
        return []

def click_restaurant(restaurant_element, index):
    """가게 클릭하여 세부 정보 패널 열기"""
    try:
        # 가게 링크 찾기
        link_element = restaurant_element.find_element(By.CSS_SELECTOR, "a.place_bluelink")
        
        # 스크롤하여 요소가 보이도록 함
        driver.execute_script("arguments[0].scrollIntoView(true);", link_element)
        time.sleep(1)
        
        # 클릭
        ActionChains(driver).move_to_element(link_element).click().perform()
        print(f"가게 {index} 클릭 완료")
        
        # 세부 정보 패널 로딩 대기
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"가게 {index} 클릭 실패: {e}")
        return False

def switch_to_detail_iframe():
    """오른쪽 세부 정보 iframe으로 전환"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 세부 정보 iframe 찾기
        detail_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
        driver.switch_to.frame(detail_iframe)
        print("세부 정보 iframe으로 전환 완료")
        return True
        
    except Exception as e:
        print(f"세부 정보 iframe 전환 실패: {e}")
        return False

def get_home_info():
    """홈 탭에서 기본 정보 수집"""
    home_info = {}
    
    try:
        # 가게 이름
        name_element = driver.find_element(By.CSS_SELECTOR, "span.GHAhO")
        home_info['name'] = name_element.text.strip()
        print(f"가게명: {home_info['name']}")
        
        # 가게 분류
        category_element = driver.find_element(By.CSS_SELECTOR, "span.lnJFt")
        home_info['category'] = category_element.text.strip()
        print(f"분류: {home_info['category']}")
        
        # 가게 주소
        try:
            address_element = driver.find_element(By.CSS_SELECTOR, "span.LDgIH")
            home_info['address'] = address_element.text.strip()
            print(f"주소: {home_info['address']}")
        except:
            home_info['address'] = "주소 정보 없음"
        
        # 전화번호 (있다면)
        try:
            phone_element = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q")
            home_info['phone'] = phone_element.text.strip()
        except:
            home_info['phone'] = "전화번호 정보 없음"
        
        # 영업시간 (있다면)
        try:
            hours_element = driver.find_element(By.CSS_SELECTOR, "time.H3ua4")
            home_info['hours'] = hours_element.text.strip()
        except:
            home_info['hours'] = "영업시간 정보 없음"
            
    except Exception as e:
        print(f"홈 정보 수집 실패: {e}")
        
    return home_info

def find_and_click_menu_tab():
    """메뉴 탭을 정확하게 찾아서 클릭"""
    try:
        print("메뉴 탭 찾는 중...")
        
        # 1단계: 모든 탭 요소들 찾기
        tab_selectors = [
            "a._tab-menu",
            "div.YYh8o a",
            "div[class*='tab'] a",
            "nav a",
            "ul li a"
        ]
        
        all_tabs = []
        for selector in tab_selectors:
            try:
                tabs = driver.find_elements(By.CSS_SELECTOR, selector)
                if tabs:
                    all_tabs = tabs
                    print(f"탭 요소들 찾음: {len(tabs)}개")
                    break
            except:
                continue
        
        if not all_tabs:
            print("탭 요소를 찾을 수 없습니다")
            return False
        
        # 2단계: 각 탭의 텍스트와 URL 확인하여 메뉴 탭 찾기
        menu_tab = None
        for i, tab in enumerate(all_tabs):
            try:
                # 탭 텍스트 확인
                tab_text = tab.text.strip().lower()
                # href 속성 확인
                tab_href = tab.get_attribute("href") or ""
                
                print(f"탭 {i+1}: 텍스트='{tab_text}', href='{tab_href}'")
                
                # 메뉴 탭인지 판단
                is_menu_tab = (
                    "menu" in tab_href.lower() or
                    "메뉴" in tab_text or
                    "menu" in tab_text
                )
                
                if is_menu_tab:
                    menu_tab = tab
                    print(f"메뉴 탭 발견: {tab_text}")
                    break
                    
            except Exception as e:
                print(f"탭 {i+1} 확인 중 오류: {e}")
                continue
        
        # 3단계: 메뉴 탭 클릭
        if menu_tab:
            try:
                # 스크롤하여 요소가 보이도록 함
                driver.execute_script("arguments[0].scrollIntoView(true);", menu_tab)
                time.sleep(1)
                
                # JavaScript로 클릭
                driver.execute_script("arguments[0].click();", menu_tab)
                time.sleep(3)
                print("메뉴 탭 클릭 완료")
                return True
                
            except Exception as e:
                print(f"메뉴 탭 클릭 실패: {e}")
                return False
        else:
            print("메뉴 탭을 찾을 수 없습니다")
            return False
            
    except Exception as e:
        print(f"메뉴 탭 찾기 실패: {e}")
        return False

def get_menu_info(menu_images_dir, restaurant_name):
    """메뉴 탭에서 메뉴 정보 수집"""
    menu_list = []
    
    try:
        # 메뉴 더보기 버튼 클릭
        try:
            more_button_selectors = [
                "a.fvwqf",
                "button.fvwqf",
                "a[class*='more']",
                "//a[contains(text(), '더보기')]"
            ]
            
            for selector in more_button_selectors:
                try:
                    if selector.startswith("//"):
                        more_button = driver.find_element(By.XPATH, selector)
                    else:
                        more_button = driver.find_element(By.CSS_SELECTOR, selector)
                    
                    if more_button.is_displayed() and more_button.is_enabled():
                        driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", more_button)
                        time.sleep(2)
                        print("메뉴 더보기 버튼 클릭 완료")
                        break
                except:
                    continue
        except:
            print("메뉴 더보기 버튼이 없거나 이미 모든 메뉴가 표시됨")
        
        # 메뉴 항목들 찾기
        menu_elements = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
        
        for i, menu_element in enumerate(menu_elements):
            menu_info = {}
            
            try:
                # 메뉴명
                name_element = menu_element.find_element(By.CSS_SELECTOR, "span.lPzHi")
                menu_info['name'] = name_element.text.strip()
                
                # 메뉴 가격
                try:
                    price_element = menu_element.find_element(By.CSS_SELECTOR, "div.GXS1X em")
                    menu_info['price'] = price_element.text.strip()
                except:
                    menu_info['price'] = "가격 정보 없음"
                
                # 메뉴 설명
                try:
                    desc_element = menu_element.find_element(By.CSS_SELECTOR, "div.TRxGt")
                    menu_info['description'] = desc_element.text.strip()
                except:
                    menu_info['description'] = "설명 없음"
                
                # 메뉴 이미지
                try:
                    img_selectors = ["img.K0PDV", "img"]
                    img_element = None
                    
                    for selector in img_selectors:
                        try:
                            img_element = menu_element.find_element(By.CSS_SELECTOR, selector)
                            break
                        except:
                            continue
                    
                    if img_element:
                        img_url = img_element.get_attribute("src")
                        if img_url:
                            # 이미지 파일명 생성 (특수문자 제거)
                            safe_menu_name = safe_filename(menu_info['name'])[:20]
                            img_filename = f"메뉴_{i+1:02d}_{safe_menu_name}.jpg"
                            
                            # 이미지 다운로드
                            downloaded_path = download_image_improved(img_url, img_filename, menu_images_dir)
                            menu_info['image_path'] = downloaded_path
                        else:
                            menu_info['image_path'] = None
                    else:
                        menu_info['image_path'] = None
                except:
                    menu_info['image_path'] = None
                
                menu_list.append(menu_info)
                print(f"메뉴 {i+1}: {menu_info['name']} - {menu_info['price']}")
                
            except Exception as e:
                print(f"메뉴 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"메뉴 정보 수집 실패: {e}")
    
    return menu_list

def find_and_click_review_tab():
    """리뷰 탭을 정확하게 찾아서 클릭"""
    try:
        print("리뷰 탭 찾는 중...")
        
        # 페이지 스크롤하여 탭이 보이도록 함
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        
        # 1단계: 모든 탭 요소들 찾기
        tab_selectors = [
            "a._tab-menu",
            "div.YYh8o a",
            "div[class*='tab'] a",
            "nav a",
            "ul li a"
        ]
        
        all_tabs = []
        for selector in tab_selectors:
            try:
                tabs = driver.find_elements(By.CSS_SELECTOR, selector)
                if tabs:
                    all_tabs = tabs
                    print(f"탭 요소들 찾음: {len(tabs)}개")
                    break
            except:
                continue
        
        if not all_tabs:
            print("탭 요소를 찾을 수 없습니다")
            return False
        
        # 2단계: 각 탭의 텍스트와 URL 확인하여 리뷰 탭 찾기
        review_tab = None
        for i, tab in enumerate(all_tabs):
            try:
                # 탭 텍스트 확인
                tab_text = tab.text.strip().lower()
                # href 속성 확인
                tab_href = tab.get_attribute("href") or ""
                
                print(f"탭 {i+1}: 텍스트='{tab_text}', href='{tab_href}'")
                
                # 리뷰 탭인지 판단
                is_review_tab = (
                    "review" in tab_href.lower() or
                    "리뷰" in tab_text or
                    "review" in tab_text
                )
                
                if is_review_tab:
                    review_tab = tab
                    print(f"리뷰 탭 발견: {tab_text}")
                    break
                    
            except Exception as e:
                print(f"탭 {i+1} 확인 중 오류: {e}")
                continue
        
        # 3단계: 리뷰 탭 클릭
        if review_tab:
            try:
                # 요소가 보이도록 스크롤
                driver.execute_script("arguments[0].scrollIntoView(true);", review_tab)
                time.sleep(1)
                
                # JavaScript로 클릭
                driver.execute_script("arguments[0].click();", review_tab)
                time.sleep(3)
                print("리뷰 탭 클릭 완료")
                return True
                
            except Exception as e:
                print(f"리뷰 탭 클릭 실패: {e}")
                return False
        else:
            print("리뷰 탭을 찾을 수 없습니다")
            return False
            
    except Exception as e:
        print(f"리뷰 탭 찾기 실패: {e}")
        return False

def click_all_review_more_buttons():
    """모든 리뷰 더보기 버튼 클릭 (개선된 버전)"""
    try:
        print("리뷰 더보기 버튼들 찾는 중...")
        
        # 지정된 셀렉터로 더보기 버튼 찾기
        more_button_selectors = [
            "div.lfH3O.fvwqf",  # 사용자가 제공한 정확한 셀렉터
            "div.fvwqf",
            "a.fvwqf",
            "button.fvwqf",
            "//a[contains(text(), '더보기')]",
            "//button[contains(text(), '더보기')]",
            "//div[contains(text(), '더보기')]"
        ]
        
        clicked_count = 0
        max_attempts = 10  # 최대 10번 시도
        
        for attempt in range(max_attempts):
            print(f"더보기 버튼 찾기 시도 {attempt + 1}/{max_attempts}")
            
            button_found = False
            
            for selector in more_button_selectors:
                try:
                    if selector.startswith("//"):
                        more_buttons = driver.find_elements(By.XPATH, selector)
                    else:
                        more_buttons = driver.find_elements(By.CSS_SELECTOR, selector)
                    
                    for button in more_buttons:
                        if button.is_displayed() and button.is_enabled():
                            try:
                                # 버튼이 보이도록 스크롤
                                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                                time.sleep(1)
                                
                                # 버튼 텍스트 확인
                                button_text = button.text.strip()
                                print(f"발견된 버튼 텍스트: '{button_text}'")
                                
                                if "더보기" in button_text or "more" in button_text.lower():
                                    # JavaScript로 클릭
                                    driver.execute_script("arguments[0].click();", button)
                                    time.sleep(3)  # 로딩 대기
                                    clicked_count += 1
                                    button_found = True
                                    print(f"더보기 버튼 클릭 완료 ({clicked_count}번째)")
                                    break
                            except Exception as click_error:
                                print(f"버튼 클릭 실패: {click_error}")
                                continue
                    
                    if button_found:
                        break
                        
                except Exception as e:
                    continue
            
            if not button_found:
                print("더 이상 더보기 버튼을 찾을 수 없음")
                break
                
            # 페이지 하단으로 스크롤하여 새로운 리뷰 로드 확인
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
        
        print(f"총 {clicked_count}개의 더보기 버튼을 클릭했습니다")
        return clicked_count > 0
        
    except Exception as e:
        print(f"리뷰 더보기 버튼 처리 실패: {e}")
        return False

def collect_review_images_with_slide(restaurant_name, review_images_dir):
    """리뷰 이미지 수집 - 프로필 영역 제외하고 방문자리뷰사진만 수집"""
    downloaded_images = []
    
    try:
        print("방문자리뷰사진만 수집 시작 (프로필 영역 제외)...")
        
        # 전체 페이지에서 alt="방문자리뷰사진"인 이미지들 찾기
        all_visitor_images = driver.find_elements(By.CSS_SELECTOR, 'img[alt="방문자리뷰사진"]')
        print(f"전체 방문자리뷰사진 발견: {len(all_visitor_images)}개")
        
        # 프로필 영역의 이미지들 제외
        profile_containers = driver.find_elements(By.CSS_SELECTOR, 'div.pui__q2fg8o.pui__A7NplK')
        profile_images = []
        
        for container in profile_containers:
            container_images = container.find_elements(By.CSS_SELECTOR, 'img')
            profile_images.extend(container_images)
        
        print(f"프로필 영역 이미지 발견: {len(profile_images)}개 (제외 예정)")
        
        total_images = 0
        excluded_count = 0
        
        for img_idx, img in enumerate(all_visitor_images):
            try:
                # 이 이미지가 프로필 영역에 속하는지 확인
                is_profile_image = False
                for profile_img in profile_images:
                    try:
                        if (img.get_attribute("src") == profile_img.get_attribute("src") or
                            driver.execute_script("return arguments[0].contains(arguments[1])", 
                                                profile_containers[profile_images.index(profile_img) // 10 if profile_images.index(profile_img) < len(profile_containers) * 10 else 0], img)):
                            is_profile_image = True
                            break
                    except:
                        continue
                
                # 프로필 영역의 이미지가 아닌 경우에만 다운로드
                if not is_profile_image:
                    img_url = img.get_attribute("src")
                    img_alt = img.get_attribute("alt")
                    
                    # alt 속성이 정확히 "방문자리뷰사진"인지 다시 한번 확인
                    if img_url and img_alt == "방문자리뷰사진":
                        filename = f"방문자리뷰_{total_images+1:03d}.jpg"
                        
                        # 이미지가 이미 다운로드되었는지 확인 (URL 기준)
                        if img_url not in [img_info.get('url') for img_info in downloaded_images]:
                            downloaded_path = download_image_improved(img_url, filename, review_images_dir)
                            if downloaded_path:
                                downloaded_images.append({
                                    'path': downloaded_path,
                                    'url': img_url,
                                    'filename': filename,
                                    'type': 'visitor_review'
                                })
                                total_images += 1
                                print(f"  방문자리뷰사진 다운로드: {filename}")
                else:
                    excluded_count += 1
                    print(f"  프로필 영역 이미지 제외: {excluded_count}개")
            
            except Exception as e:
                print(f"  이미지 {img_idx+1} 처리 실패: {e}")
                continue
        
        print(f"방문자리뷰사진 다운로드 완료: 총 {total_images}개")
        print(f"프로필 영역 이미지 제외: {excluded_count}개")
        return [img['path'] for img in downloaded_images]
        
    except Exception as e:
        print(f"방문자리뷰사진 수집 실패: {e}")
        return []

def get_review_info(review_images_dir, restaurant_name, max_reviews=None):
    """리뷰 탭에서 리뷰 정보 수집 (개선된 버전)"""
    if max_reviews is None:
        max_reviews = MAX_REVIEWS_PER_RESTAURANT
    
    review_list = []
    
    try:
        # 모든 리뷰 더보기 버튼 클릭
        click_all_review_more_buttons()
        
        # 리뷰 이미지 수집 (슬라이드 포함)
        review_slide_images = collect_review_images_with_slide(restaurant_name, review_images_dir)
        
        # 최종 스크롤하여 모든 리뷰 로드
        print("최종 스크롤로 모든 리뷰 로드 중...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # 리뷰 항목들 찾기 (여러 셀렉터 시도)
        review_selectors = [
            "li.place_apply_pui.EjjAW",
            "li.place_apply_pui",
            "div.pui__vn15t2",
            "li[class*='review']",
            "div[class*='review']"
        ]
        
        review_elements = []
        for selector in review_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements:
                    review_elements = elements[:max_reviews]
                    print(f"리뷰 요소 찾음: {selector} - {len(review_elements)}개")
                    break
            except:
                continue
        
        if not review_elements:
            print("리뷰 요소를 찾을 수 없습니다.")
            return review_list
        
        print(f"총 {len(review_elements)}개의 리뷰 처리 시작")
        
        for i, review_element in enumerate(review_elements):
            review_info = {}
            
            try:
                # 리뷰 텍스트 - 정확한 셀렉터 사용
                review_text = "리뷰 텍스트 없음"
                try:
                    # div.pui__vn15t2에서 리뷰 텍스트 찾기
                    text_element = review_element.find_element(By.CSS_SELECTOR, "div.pui__vn15t2")
                    review_text = text_element.text.strip()
                    if not review_text or len(review_text) < 5:
                        # 다른 텍스트 셀렉터들도 시도
                        text_selectors = [
                            "span.zPfVt", 
                            "div.pui__vn15t2 span",
                            "span[class*='review']", 
                            "div[class*='content']"
                        ]
                        
                        for text_selector in text_selectors:
                            try:
                                text_elements = review_element.find_elements(By.CSS_SELECTOR, text_selector)
                                for text_elem in text_elements:
                                    text_content = text_elem.text.strip()
                                    if text_content and len(text_content) > 10:
                                        review_text = text_content
                                        break
                                if review_text != "리뷰 텍스트 없음":
                                    break
                            except:
                                continue
                except:
                    # 여전히 텍스트를 못찾았다면 전체 텍스트에서 추출 시도
                    try:
                        full_text = review_element.text.strip()
                        if len(full_text) > 20:
                            sentences = full_text.split('\n')
                            for sentence in sentences:
                                if len(sentence.strip()) > 10:
                                    review_text = sentence.strip()
                                    break
                    except:
                        pass
                
                review_info['text'] = review_text
                
                # 리뷰 날짜 - 두 번째 span.pui__blind 요소에서 실제 날짜 추출
                review_date = "날짜 정보 없음"
                try:
                    # span.pui__gfuUIT 안의 모든 span.pui__blind 요소들 찾기
                    date_container = review_element.find_element(By.CSS_SELECTOR, "span.pui__gfuUIT")
                    date_elements = date_container.find_elements(By.CSS_SELECTOR, "span.pui__blind")
                    
                    # 두 번째 span.pui__blind 요소에서 날짜 추출 (첫 번째는 "방문일"이므로 제외)
                    if len(date_elements) >= 2:
                        date_text = date_elements[1].text.strip()  # 두 번째 요소 사용
                        
                        if date_text:
                            # 날짜 패턴 추출
                            date_patterns = [
                                r'(\d{4}년\s*\d{1,2}월\s*\d{1,2}일)',  # 2025년 5월 15일
                                r'(\d{4}\.\d{1,2}\.\d{1,2})',         # 2024.01.15
                                r'(\d{4}-\d{1,2}-\d{1,2})',          # 2024-01-15
                                r'(\d{1,2}\.\d{1,2})',               # 01.15
                                r'(\d+일전)',                         # 3일전
                                r'(\d+주전)',                         # 2주전
                                r'(\d+개월전)',                       # 1개월전
                                r'(\d+년전)',                         # 1년전
                                r'(오늘)',                            # 오늘
                                r'(어제)'                             # 어제
                            ]
                            
                            for pattern in date_patterns:
                                date_match = re.search(pattern, date_text)
                                if date_match:
                                    review_date = date_match.group(1)
                                    break
                            
                            # 패턴에 매칭되지 않는 경우 전체 텍스트 사용 (요일 포함된 경우 등)
                            if review_date == "날짜 정보 없음" and len(date_text) < 50:
                                review_date = date_text
                    
                    elif len(date_elements) == 1:
                        # span.pui__blind가 하나뿐인 경우, 그것이 날짜일 가능성
                        date_text = date_elements[0].text.strip()
                        if date_text and "방문일" not in date_text:
                            review_date = date_text
                        
                except:
                    # 대체 날짜 셀렉터들 시도
                    date_selectors = [
                        "time", 
                        "div.pui__QKE5Pr", 
                        "span[class*='date']",
                        "div[class*='date']",
                        "span.time"
                    ]
                    
                    for date_selector in date_selectors:
                        try:
                            date_element = review_element.find_element(By.CSS_SELECTOR, date_selector)
                            date_text = date_element.text.strip()
                            
                            if date_text and len(date_text) < 30 and "방문일" not in date_text:
                                review_date = date_text
                                break
                        except:
                            continue
                
                review_info['date'] = review_date
                
                # 리뷰 평점 (별점) 수집
                try:
                    rating_selectors = [
                        "div.pui__rating span",
                        "span[class*='rating']",
                        "div[class*='star']",
                        ".rating"
                    ]
                    
                    review_rating = "평점 정보 없음"
                    for rating_selector in rating_selectors:
                        try:
                            rating_element = review_element.find_element(By.CSS_SELECTOR, rating_selector)
                            rating_text = rating_element.text.strip()
                            if rating_text and ("점" in rating_text or "★" in rating_text):
                                review_rating = rating_text
                                break
                        except:
                            continue
                    
                    review_info['rating'] = review_rating
                except:
                    review_info['rating'] = "평점 정보 없음"
                
                # 리뷰 이미지들 (개별 리뷰에서) - 프로필 영역 제외
                review_images = []
                try:
                    # 현재 리뷰 요소 내의 프로필 영역 찾기
                    profile_containers = review_element.find_elements(By.CSS_SELECTOR, 'div.pui__q2fg8o.pui__A7NplK')
                    profile_images = []
                    
                    for container in profile_containers:
                        container_images = container.find_elements(By.CSS_SELECTOR, 'img')
                        profile_images.extend(container_images)
                    
                    # alt="방문자리뷰사진"인 이미지만 찾기
                    visitor_images = review_element.find_elements(By.CSS_SELECTOR, 'img[alt="방문자리뷰사진"]')
                    
                    for j, img_element in enumerate(visitor_images):
                        # 이 이미지가 프로필 영역에 속하는지 확인
                        is_profile_image = False
                        for profile_img in profile_images:
                            try:
                                if img_element.get_attribute("src") == profile_img.get_attribute("src"):
                                    is_profile_image = True
                                    break
                            except:
                                continue
                        
                        # 프로필 이미지가 아닌 경우에만 다운로드
                        if not is_profile_image:
                            img_url = img_element.get_attribute("src")
                            img_alt = img_element.get_attribute("alt")
                            
                            # alt 속성이 정확히 "방문자리뷰사진"인지 다시 한번 확인
                            if img_url and img_alt == "방문자리뷰사진":
                                img_filename = f"개별방문자리뷰_{i+1:02d}_{j+1:02d}.jpg"
                                downloaded_path = download_image_improved(img_url, img_filename, review_images_dir)
                                if downloaded_path:
                                    review_images.append(downloaded_path)
                        
                except Exception as e:
                    print(f"리뷰 {i+1} 방문자리뷰사진 수집 실패: {e}")
                
                review_info['individual_images'] = review_images
                review_list.append(review_info)
                
                # 진행 상황 출력
                if i % 10 == 0:
                    print(f"리뷰 처리 진행률: {i+1}/{len(review_elements)}")
                
                print(f"리뷰 {i+1}: {review_info['text'][:50]}..." if len(review_info['text']) > 50 else f"리뷰 {i+1}: {review_info['text']}")
                
            except Exception as e:
                print(f"리뷰 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"리뷰 정보 수집 실패: {e}")
        
    print(f"총 {len(review_list)}개의 리뷰 수집 완료")
    return review_list

def go_back_to_list():
    """목록으로 돌아가기"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 검색 결과 iframe으로 다시 전환
        search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
        driver.switch_to.frame(search_iframe)
        print("목록으로 돌아가기 완료")
        return True
        
    except Exception as e:
        print(f"목록으로 돌아가기 실패: {e}")
        return False

# =================== 메인 실행 부분 ===================
try:
    print("=" * 60)
    print(f"네이버 지도 크롤링 시작")
    print(f"검색어: {SEARCH_KEYWORD}")
    print(f"최대 가게 수: {MAX_RESTAURANTS}")
    print(f"가게당 최대 리뷰 수: {MAX_REVIEWS_PER_RESTAURANT}")
    print("=" * 60)
    
    # 1. 검색어 설정 및 접속
    encoded_keyword = urllib.parse.quote(SEARCH_KEYWORD)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. 디렉토리 생성
    base_dir = create_directory_structure(SEARCH_KEYWORD)
    print(f"저장 디렉토리 생성: {base_dir}")
    
    # 3. 검색 결과 iframe 접근
    print("검색 결과 iframe으로 전환 중...")
    search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(search_iframe)
    
    # 4. 가게 리스트 수집
    restaurant_elements = get_restaurant_list()
    total_restaurants = len(restaurant_elements)
    print(f"총 {total_restaurants}개 가게 발견")
    
    # 5. 각 가게별로 세부 정보 수집
    max_restaurants = min(MAX_RESTAURANTS, total_restaurants)
    
    for i in range(max_restaurants):
        print(f"\n{'='*20} 가게 {i+1}/{max_restaurants} {'='*20}")
        
        try:
            # 가게 클릭
            if not click_restaurant(restaurant_elements[i], i+1):
                continue
            
            # 세부 정보 iframe으로 전환
            if not switch_to_detail_iframe():
                continue
            
            # 홈 정보 수집
            print("홈 정보 수집 중...")
            home_info = get_home_info()
            
            # 가게별 폴더 구조 생성
            restaurant_dir, menu_images_dir, review_images_dir = create_restaurant_directories(
                base_dir, home_info.get('name', f'restaurant_{i+1}')
            )
            
            # 메뉴 정보 수집
            menu_info = []
            if find_and_click_menu_tab():
                print("메뉴 정보 수집 중...")
                menu_info = get_menu_info(menu_images_dir, home_info.get('name', f'restaurant_{i+1}'))
            else:
                print("메뉴 탭을 찾을 수 없어 메뉴 정보 수집을 건너뜁니다.")
            
            # 리뷰 정보 수집
            review_info = []
            if find_and_click_review_tab():
                print("리뷰 정보 수집 중...")
                review_info = get_review_info(review_images_dir, home_info.get('name', f'restaurant_{i+1}'))
            else:
                print("리뷰 탭을 찾을 수 없어 리뷰 정보 수집을 건너뜁니다.")
            
            # 전체 정보 결합
            restaurant_data = {
                'index': i + 1,
                'home_info': home_info,
                'menu_info': menu_info,
                'review_info': review_info,
                'restaurant_dir': restaurant_dir,
                'collected_at': datetime.now().isoformat()
            }
            
            all_restaurant_data.append(restaurant_data)
            print(f"가게 {i+1} 정보 수집 완료!")
            print(f"- 메뉴 개수: {len(menu_info)}")
            print(f"- 리뷰 개수: {len(review_info)}")
            
            # 목록으로 돌아가기
            go_back_to_list()
            time.sleep(2)
            
        except Exception as e:
            print(f"가게 {i+1} 처리 중 오류: {e}")
            # 목록으로 돌아가기 시도
            try:
                go_back_to_list()
            except:
                pass
            continue
    
    # 6. 결과 저장
    print(f"\n{'='*60}")
    print(f"총 {len(all_restaurant_data)}개 가게 상세 정보 수집 완료!")
    
    # JSON 파일 저장
    safe_keyword = safe_filename(SEARCH_KEYWORD)
    json_filename = os.path.join(base_dir, f"{safe_keyword}_detailed_restaurants.json")
    with open(json_filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
    
    # CSV 파일 저장 (한글 헤더로 개선)
    csv_filename = os.path.join(base_dir, f"{safe_keyword}_restaurants_summary.csv")
    with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:  # utf-8-sig로 BOM 추가
        fieldnames = ['순번', '가게명', '분류', '주소', '전화번호', '메뉴수', '리뷰수', '폴더경로']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for restaurant in all_restaurant_data:
            row = {
                '순번': restaurant['index'],
                '가게명': restaurant['home_info'].get('name', ''),
                '분류': restaurant['home_info'].get('category', ''),
                '주소': restaurant['home_info'].get('address', ''),
                '전화번호': restaurant['home_info'].get('phone', ''),
                '메뉴수': len(restaurant['menu_info']),
                '리뷰수': len(restaurant['review_info']),
                '폴더경로': restaurant.get('restaurant_dir', '')
            }
            writer.writerow(row)
    
    # 리뷰 상세 정보 CSV 저장
    reviews_csv_filename = os.path.join(base_dir, f"{safe_keyword}_reviews_detail.csv")
    with open(reviews_csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['가게명', '리뷰번호', '리뷰내용', '작성날짜', '평점', '이미지수']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for restaurant in all_restaurant_data:
            restaurant_name = restaurant['home_info'].get('name', '')
            for idx, review in enumerate(restaurant['review_info']):
                row = {
                    '가게명': restaurant_name,
                    '리뷰번호': idx + 1,
                    '리뷰내용': review.get('text', ''),
                    '작성날짜': review.get('date', ''),
                    '평점': review.get('rating', ''),
                    '이미지수': len(review.get('individual_images', []))
                }
                writer.writerow(row)
    
    # 통계 정보 출력
    total_menus = sum(len(restaurant['menu_info']) for restaurant in all_restaurant_data)
    total_reviews = sum(len(restaurant['review_info']) for restaurant in all_restaurant_data)
    total_images = 0
    
    # 이미지 파일 개수 세기
    for restaurant in all_restaurant_data:
        restaurant_dir = restaurant.get('restaurant_dir', '')
        if os.path.exists(restaurant_dir):
            for root, dirs, files in os.walk(restaurant_dir):
                image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
                total_images += len(image_files)
    
    print(f"\n파일 저장 완료:")
    print(f"- 상세 정보 (JSON): {json_filename}")
    print(f"- 요약 정보 (CSV): {csv_filename}")
    print(f"- 리뷰 상세 (CSV): {reviews_csv_filename}")
    print(f"- 이미지 폴더: 각 가게별 폴더")
    print(f"\n수집 통계:")
    print(f"- 총 가게 수: {len(all_restaurant_data)}")
    print(f"- 총 메뉴 수: {total_menus}")
    print(f"- 총 리뷰 수: {total_reviews}")
    print(f"- 총 이미지 수: {total_images}")
    
    print(f"\n{'='*60}")
    print("크롤링 완료!")

except Exception as e:
    print(f"전체 프로세스 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    try:
        pass
#         driver.quit()
#         print("웹드라이버 종료 완료")
    except:
        pass

네이버 지도 크롤링 시작
검색어: 신림동 햄버거
최대 가게 수: 1
가게당 최대 리뷰 수: 10
네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%ED%96%84%EB%B2%84%EA%B1%B0
저장 디렉토리 생성: 신림동 햄버거_테스트(1개)_data
검색 결과 iframe으로 전환 중...
가게 리스트 수집 중...
스크롤 컨테이너 찾음
현재 로드된 가게 수: 10
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
더 이상 로드할 가게가 없음
총 50개 가게 발견

==================== 가게 1/1 ====================
가게 1 클릭 완료
세부 정보 iframe으로 전환 완료
홈 정보 수집 중...
가게명: 아토커피 신림점
분류: 카페,디저트
주소: 서울 관악구 관천로 79 1층
메뉴 탭 찾는 중...
탭 요소들 찾음: 6개
탭 1: 텍스트='홈', href='https://pcmap.place.naver.com/restaurant/1539251371/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202507081918&locale=ko&svcName=map_pcv5&searchText=%EC%8B%A0%EB%A6%BC%EB%8F%99%20%ED%96%84%EB%B2%84%EA%B1%B0'
탭 2: 텍스트='소식', href='https://pcmap.place.naver.com/restaurant/1539251371/feed?entry=bmp&from=map&fromPanelNum=2&timestamp=202507081918&locale=ko&svcName=map_pcv5&searchText=%EC%8B%A0%EB%A6%BC%EB%8F%99%20%ED%96%84%EB%B2%84%EA%B1%B0'
탭 3: 텍스트='메뉴', href='https://pcmap.plac

  이미지 다운로드 성공: 방문자리뷰_011.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_011.jpg
  이미지 다운로드 성공: 방문자리뷰_012.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_012.jpg
  이미지 다운로드 성공: 방문자리뷰_013.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_013.jpg
  이미지 다운로드 성공: 방문자리뷰_014.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_014.jpg
  이미지 다운로드 성공: 방문자리뷰_015.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_015.jpg
  이미지 다운로드 성공: 방문자리뷰_016.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_016.jpg
  이미지 다운로드 성공: 방문자리뷰_017.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_017.jpg
  이미지 다운로드 성공: 방문자리뷰_018.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_018.jpg
  이미지 다운로드 성공: 방문자리뷰_019.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_019.jpg
  이미지 다운로드 성공: 방문자리뷰_020.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_020.jpg
  이미지 다운로드 성공: 방문자리뷰_021.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_021.jpg
  이미지 다운로드 성공: 방문자리뷰_022.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_022.jpg
  이미지 다운로드 성공: 방문자리뷰_023.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_023.jpg
  이미지 다운로드 성공: 방문자리뷰_024.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_024.jpg
  이미지 다운로드 성공: 방문자리뷰_025.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_025.jpg
  이미지 다운로드 성공: 방문자리뷰_026.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_026.jpg
  이미지 다운로드 성공: 방문자리뷰_027.jpg
  방문자리뷰사진 다운로드: 방문자리뷰_027.j